In [20]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = OpenAI(temperature=0)

In [21]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [22]:
from langchain.document_loaders import TextLoader

loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [23]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [24]:
from langchain.document_loaders import WebBaseLoader

In [25]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [26]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 1354, which is longer than the specified 1000
Created a chunk of size 1923, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000


In [27]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

In [28]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [29]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [31]:
agent.run(
    "What did Biden say about taxes in the latest SotU speech?"
)



> Entering new  chain...
 I need to find out what Biden said about taxes in the latest SotU speech.
Action: State of Union QA System
Action Input: What did Biden say about taxes in the latest SotU speech?
Observation:  Biden said that he has proposed closing loopholes so the very wealthy don't pay a lower tax rate than a teacher or a firefighter, and he has proposed a 15% minimum tax rate for corporations. He also said that nobody earning less than $400,000 a year will pay an additional penny in new taxes.
Thought: I now know the final answer
Final Answer: Biden proposed closing loopholes so the very wealthy don't pay a lower tax rate than a teacher or a firefighter, and he proposed a 15% minimum tax rate for corporations. He also said that nobody earning less than $400,000 a year will pay an additional penny in new taxes.

> Finished chain.


"Biden proposed closing loopholes so the very wealthy don't pay a lower tax rate than a teacher or a firefighter, and he proposed a 15% minimum tax rate for corporations. He also said that nobody earning less than $400,000 a year will pay an additional penny in new taxes."

In [32]:
agent.run("When would we not want to use ruff?")



> Entering new  chain...
 I need to find out what ruff is and when it should not be used.
Action: Ruff QA System
Action Input: When would we not want to use ruff?
Observation:  You would not want to use Ruff if you need to use custom lint rules or if you are using Python 2 code.
Thought: I now know the final answer
Final Answer: You would not want to use Ruff if you need to use custom lint rules or if you are using Python 2 code.

> Finished chain.


'You would not want to use Ruff if you need to use custom lint rules or if you are using Python 2 code.'

In [13]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [14]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [15]:
agent.run(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new  chain...
 I need to find out what Biden said about Ketanji Brown Jackson in the State of the Union address.
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the State of the Union address?
Observation:  Biden said that Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.


> Finished chain.


" Biden said that Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

In [16]:
agent.run("Why use ruff over flake8?")



> Entering new  chain...
 I need to find out the advantages of using ruff over flake8
Action: Ruff QA System
Action Input: What are the advantages of using ruff over flake8?
Observation:  Ruff can be used as a drop-in replacement for Flake8 when used (1) without or with a small number of plugins, (2) alongside Black, and (3) on Python 3 code. Ruff also re-implements some of the most popular Flake8 plugins and related code quality tools natively, and it can automatically fix its own lint violations. Additionally, Ruff uses different rule codes and prefixes than would be found in the originating Flake8 plugins, which helps minimize conflicts across plugins and allows any individual plugin to be toggled on or off with a single (e.g.) --select TID, as opposed to --select I2.


> Finished chain.


' Ruff can be used as a drop-in replacement for Flake8 when used (1) without or with a small number of plugins, (2) alongside Black, and (3) on Python 3 code. Ruff also re-implements some of the most popular Flake8 plugins and related code quality tools natively, and it can automatically fix its own lint violations. Additionally, Ruff uses different rule codes and prefixes than would be found in the originating Flake8 plugins, which helps minimize conflicts across plugins and allows any individual plugin to be toggled on or off with a single (e.g.) --select TID, as opposed to --select I2.'

In [17]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [18]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [19]:
agent.run(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new  chain...
 I need to find out what tool ruff uses to run over Jupyter Notebooks, and if the president mentioned it in the state of the union.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: I need to find out if the president mentioned nbQA in the state of the union.
Action: State of Union QA System
Action Input: Did the president mention nbQA in the state of the union?
Observation:  No, the president did not mention nbQA in the state of the union.
Thought: I now know the final answer.
Final Answer: Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks, but the president did not mention nbQA in the state of the union.

> Finished chain.


'Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks, but the president did not mention nbQA in the state of the union.'